<a href="https://colab.research.google.com/github/mahmoudrodeiny/rodeiny/blob/master/match_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!sudo apt-get install -y tesseract-ocr
!pip install -q ultralytics opencv-python pytesseract scikit-learn matplotlib


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ball_tiled_output.mp4 to ball_tiled_output (1).mp4


In [ ]:
import cv2
import numpy as np
import pytesseract
from ultralytics import YOLO
from sklearn.cluster import KMeans
from collections import Counter


pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
custom_config = r'--oem 3 --psm 6 outputbase digits'

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
video_path = '/content/ball_tiled_output (1).mp4'
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('output_clustered_teams.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))


In [ ]:
def get_dominant_color(image, k=2):
    image = cv2.resize(image, (50, 50))
    img = image.reshape((-1, 3))
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(img)
    colors = kmeans.cluster_centers_.astype(int)
    counts = Counter(kmeans.labels_)
    dominant = colors[np.argmax(list(counts.values()))]
    return tuple(int(c) for c in dominant)

def classify_team_by_color(color):
    r, g, b = color
    if r > 150 and g < 100 and b < 100:
        return "Team Red"
    elif r > 200 and g > 200 and b > 200:
        return "Team White"
    else:
        return "Other"


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)
    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls[0])
            label = model.names[cls_id]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            roi = frame[y1:y2, x1:x2]

            if label == 'person':
                color = get_dominant_color(roi)
                team = classify_team_by_color(color)


                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                jersey_number = pytesseract.image_to_string(gray, config=custom_config).strip()
                if jersey_number:
                    team += f" #{jersey_number}"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color[::-1], 2)
                cv2.putText(frame, team, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color[::-1], 2)

            elif label == 'ball':
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, 'Ball', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("Processed video saved as: output.mp4")


Processed video saved as: output.mp4
